In [18]:
import torch
import pandas as pd
import numpy as np

# 读取../data/train_data.csv文件
train_df = pd.read_csv('../data/train_data.csv')
# val_df = pd.read_csv('../data/test_data.csv')


train_df['dteday'] = pd.to_datetime(train_df['dteday'])

# # 提取年、月、日，添加为新的整数字段
# train_df['year'] = train_df['dteday'].dt.year
# train_df['month'] = train_df['dteday'].dt.month
# train_df['day'] = train_df['dteday'].dt.day


# val_df['dteday'] = pd.to_datetime(val_df['dteday'])

# # 提取年、月、日，添加为新的整数字段
# val_df['year'] = val_df['dteday'].dt.year
# val_df['month'] = val_df['dteday'].dt.month
# val_df['day'] = val_df['dteday'].dt.day

# 删除date列
# train_df.drop('dteday', axis=1, inplace=True)
# val_df.drop('dteday', axis=1, inplace=True)


# 查看数据集的前几行，并打印数据集的基本信息

print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15219 entries, 0 to 15218
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   instant     15216 non-null  float64       
 1   dteday      15216 non-null  datetime64[ns]
 2   season      15216 non-null  float64       
 3   yr          15216 non-null  float64       
 4   mnth        15216 non-null  float64       
 5   hr          15216 non-null  float64       
 6   holiday     15216 non-null  float64       
 7   weekday     15216 non-null  float64       
 8   workingday  15216 non-null  float64       
 9   weathersit  15216 non-null  float64       
 10  temp        15216 non-null  float64       
 11  atemp       15216 non-null  float64       
 12  hum         15216 non-null  float64       
 13  windspeed   15216 non-null  float64       
 14  casual      15216 non-null  float64       
 15  registered  15216 non-null  float64       
 16  cnt         15216 non-

In [19]:
train_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1.0,2011-01-01,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,8.0,32.0,40.0
2,3.0,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,5.0,27.0,32.0
3,4.0,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0,3.0,10.0,13.0
4,5.0,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0,0.0,1.0,1.0


In [36]:


categorical_covariates = ['season','yr','mnth','hr','holiday','weekday','workingday','weathersit']

categorical_covariates_num_embeddings = [[],[]]
for col in categorical_covariates:
    categorical_covariates_num_embeddings[0].append(train_df[col].nunique())
    # categorical_covariates_num_embeddings[1].append(val_df[col].nunique())

numeric_covariates = ['temp','atemp','hum','windspeed','casual','registered','cnt']
target_idx = np.where(np.array(numeric_covariates)=='cnt')[0][0]



# ,,,,,,,,,

In [37]:
train_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1.0,2011-01-01,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,8.0,32.0,40.0
2,3.0,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,5.0,27.0,32.0
3,4.0,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0,3.0,10.0,13.0
4,5.0,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0,0.0,1.0,1.0


In [38]:
def dataframe_to_tensor(series,numeric_covariates,categorical_covariates,target_idx):

    numeric_cov_arr = np.array([series[numeric_covariates].values.tolist()])
    category_cov_arr = np.array([series[categorical_covariates].values.tolist()])
    # static_cov_arr = np.array(series[categorical_static].values.tolist())

    x_numeric = torch.tensor(numeric_cov_arr,dtype=torch.float32)#.transpose(2,1)
    x_numeric = torch.log(x_numeric+1e-5)
    x_category = torch.tensor(category_cov_arr,dtype=torch.long)#.transpose(2,1)
    # x_static = torch.tensor(static_cov_arr,dtype=torch.long)
    y = torch.tensor(numeric_cov_arr[:,:,target_idx],dtype=torch.float32)

    return x_numeric, x_category, y


window_size = 16
forecast_length = 16
num_val = 2

val_max_date = '2012/10/1'
train_max_date = str((pd.to_datetime(val_max_date) - pd.Timedelta(days=window_size*num_val+forecast_length)).date())

train_final = train_df[train_df['dteday']<=train_max_date]
val_final = train_df[(train_df['dteday']>train_max_date)&(train_df['dteday']<=val_max_date)]
train_final = train_df
# val_final = val_df

# train_series = train_final.groupby(categorical_static+['family']).agg(list).reset_index()
# val_series = val_final.groupby(categorical_static+['family']).agg(list).reset_index()

x_numeric_train_tensor, x_category_train_tensor, y_train_tensor = dataframe_to_tensor(train_df,numeric_covariates,categorical_covariates,target_idx)

# x_numeric_val_tensor, x_category_val_tensor, y_val_tensor = dataframe_to_tensor(val_df,numeric_covariates,categorical_covariates,target_idx)

In [39]:
x_category_train_tensor[0,0,:]

tensor([1, 0, 1, 0, 0, 6, 0, 1])

In [40]:
y_train_tensor[0,1]

tensor(40.)